In [1]:
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import imageio
import numpy as np
import cv2
from torchvision.transforms.functional import gaussian_blur

data = torch.load(r"./data/n_512_dt_0.1_F_512/cloud/000000.pt", weights_only=True)

In [2]:
WIDTH = 512
HEIGHT = 512

In [ ]:
def points_to_field(X, weight, batch_size=4):
    

In [4]:
X = data["X"]
F, n, _ = X.shape

m = torch.broadcast_to(data["m"][None,:], (F, n))
print(X.shape, m.shape)

torch.Size([512, 512, 2]) torch.Size([512, 512])


In [5]:
hist = points_to_histograms(X, m, batch_size=64)
hist = gaussian_blur(hist, kernel_size=11)

100%|██████████| 8/8 [00:00<00:00, 116.37it/s]


In [6]:
x_min = 0
x_max = 512
y_min = 0
y_max = 512

frames = []
fig, ax = plt.subplots(figsize=(5.12, 5.12), dpi=100)
fig.patch.set_facecolor('black')

F, _, _ = hist.shape

for i in tqdm(range(F), ncols=80):
    ax.clear()
    ax.set_xlim(0, 512)
    ax.set_ylim(0, 512)
    ax.imshow(hist[i], cmap="Greys_r", vmin=0, vmax=0.5)
    plt.axis("off")
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.buffer_rgba(), dtype="uint8")
    image = image.reshape(fig.canvas.get_width_height() + (4,))
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    
    # Make the image writable by creating a copy
    image = image.copy()
    frames.append(image)

plt.close()

print(f"Finished rendering, saving to MP4...")

# Save frames as an animated GIF with looping 
imageio.mimsave(f"./test.mp4", frames, fps=30) #, loop=0)

100%|█████████████████████████████████████████| 512/512 [00:14<00:00, 35.59it/s]


Finished rendering, saving to MP4...
